In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split
from PIL import Image
from torchvision.datasets import ImageFolder

In [ ]:
# Define the root directories for training and testing
train_dir = 'C:\\Users\\Patrik\\Desktop\\Files\\fei\\Ing\\2nd class\\HNS\\Dataset_Znacky\\Train-adj'
test_dir = 'C:\\Users\\Patrik\\Desktop\\Files\\fei\\Ing\\2nd class\\HNS\\Dataset_Znacky\\test-no-labels'
num_classes = len(os.listdir(train_dir))
num_train_images_per_class = 800
num_test_images_per_class = 200

# Create empty lists to store training and testing datasets
train_dataset = []
test_dataset = []

# Loop through each subfolder (class) in the training directory
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)

    if os.path.isdir(class_dir):
        images = os.listdir(class_dir)
        random.shuffle(images)
        train_dataset.extend([(os.path.join(class_dir, img), int(class_name)) for img in images[:num_train_images_per_class]])

# Loop through each subfolder (class) in the testing directory
for class_name in os.listdir(test_dir):
    class_dir = os.path.join(test_dir, class_name)

    if os.path.isdir(class_dir):
        images = os.listdir(class_dir)
        random.shuffle(images)
        test_dataset.extend([(os.path.join(class_dir, img), int(class_name)) for img in images[:num_test_images_per_class]])



In [ ]:
# Define transforms for data augmentation and normalization
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define your custom dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path)  # Open the image as a PIL Image

        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# Create the custom datasets
train_dataset = CustomDataset(train_dataset, data_transforms)
test_dataset = CustomDataset(test_dataset, data_transforms)

# Split the datasets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Load the pre-trained AlexNet model
model = models.alexnet(pretrained=True)

# Update the output layer to match the number of classes in your dataset
num_classes = 34  # Replace with the actual number of classes in your dataset
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, num_classes)

# Set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Training loop
num_epochs = 10
device = torch.device("cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

In [ ]:
# Validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = 100 * correct / total
print(f"Validation Accuracy: {val_accuracy:.2f}%")